In [ ]:
import pandas as pd
import re 
from fuzzywuzzy import process
import os

## Read all the sheets of the input workbook
dfs = pd.read_excel("E:\myport\input\MyPoert AUG sheet1.xlsx", sheet_name=None)


# Remove excessive columns.
# Unify the column names. 
# Add a country column using the dictinary key of (dfs).
for key, value in dfs.items():
    dfs[key]= dfs[key].iloc[:, 0:3]
    dfs[key].columns = ['trip', 'price_offer', 'country']
    dfs[key]['country'] = str(key)



# ADD all sheets into one dataframe
all_dfs = list(dfs.values())
full_data = pd.concat(all_dfs, ignore_index=True)



# Read backend files which will validate the main file manipulation
countries = pd.read_excel('E:\myport\input\Countries IDs.xlsx')
ports = pd.read_excel('E:\myport\input\Ports_ceties_codes_and_IDs.xlsx')

full_data['price_offer2'] = full_data['price_offer'].shift(-1)
full_data['price_offer3'] = full_data['price_offer'].shift(-2)
full_data = full_data [ full_data ['trip'].notnull()]
full_data0 = full_data

# Extract departure port, arrival port, trip type, And price
def extract_from_to_trip_type(row):
    """
    Extracts the departure and destination of the trip from 'trip' column,
    Price and trip_type from 'price_offer' column.
    """
    
    # Setting the initial values of cells.
    trip = row['trip']
    price_offer = row['price_offer']
    price_offer2 = row['price_offer2']
    price_offer3 = row['price_offer3']
    
    # Extracting the departure and destination ports from the trip column using regex
    search = re.findall(r'(?:FRM|From|from|FROM)(?:\s*)(.*?)(?:\s*to|TO|To)(?:\s*)(.*?)\s*$', trip)
    if search:
        full_from, full_to = search[0]
    else:
        full_from, full_to = None, None

    # Extracting trip_type from price_offer column
    trip_type_match = re.search(r'(?:Trip|TRIP|trip)(?:\s?)(?::)(?:\s?)(\w)', price_offer)
    trip_type = trip_type_match.group(1).upper() if trip_type_match else None

    # Extracting price from price_offer column
    price_match = re.search(r'(\d+)', price_offer)
    if price_match:
        price = price_match.group(1)
    else:
        price_match = re.search(r'(\d+)', price_offer2)
        if price_match:
            price = price_match.group(1)
        else:
            price_match = re.search(r'(\d+)', price_offer3)
            price = price_match.group(1)

    return pd.Series({'full_from': full_from, 'full_to': full_to, 'trip_type': trip_type, 'price_raw': price})

# Append the extracted values to the data frame
full_data[['full_from', 'full_to', 'trip_type', 'price_raw']] = full_data.apply(extract_from_to_trip_type, axis=1)
full_data1 = full_data

# create a column containing a price dictionary that also includes Arabic script 
def ar_num(row):
    p_raw = row['price_raw']
    
    ar_num = '۰١٢٣٤٥٦٧٨٩'
    en_num = '0123456789'

    translator = str.maketrans(en_num, ar_num)
    p_ar = p_raw.translate(translator)

    return '{' + f'"en":"{p_raw}","ar":"{p_ar}","gr":"{p_raw}","it":"{p_raw}","cz":"{p_raw}","fr":"{p_raw}","sk":"{p_raw}"' + '}'

full_data['price'] = full_data.apply(ar_num, axis=1)



# Drop unneded columns 
full_data.drop(['trip', 'price_offer', 'price_offer2', 'price_offer3', 'price_raw'], axis=1, inplace=True)



# Read a global country/capital CSV to compensate entries with capital mentioned instead of country
country_capital = pd.read_csv('https://raw.githubusercontent.com/icyrockcom/country-capitals/master/data/country-list.csv', usecols=["country", "capital"])

# Merge the manipulated data with our countries file from backend
# Then merge again with capital from github file.
full_data = full_data.merge(countries, how='left', left_on='country', right_on='Country')
full_data = full_data.merge(country_capital, how='left', left_on='country', right_on='capital')
full_data.fillna('', inplace=True)
full_data2 = full_data


# Create a column named corrected_country with the right country name for a better merge with the file from the backend.
def country_correction(row):
    country1 = row['Country']
    country2 = row['country_y']

    if country1 == '':
        corrected_country = country2
    else:
        corrected_country = country1

    return corrected_country

full_data['corrected_country'] = full_data.apply(country_correction, axis=1)
full_data3 = full_data

# Drop unneded columns
full_data = full_data.drop(["country_x", "ID", "Country", "country_y", "capital"], axis=1)
full_data = full_data.merge(countries, how='left', left_on='corrected_country', right_on='Country')
full_data4 = full_data

# Add publish_date and expire_date columns
# Create the 'publish_date' column with a fixed value
full_data['publish_date'] = '04-04-2024'

# Convert the 'publish_date' column to datetime format
full_data['publish_date'] = pd.to_datetime(full_data['publish_date'], format='%d-%m-%Y')

# Calculate the 'expire_date' by adding 33 days to the 'publish_date'
full_data['expire_date'] = full_data['publish_date'] + pd.Timedelta(days=33)

# Convert date columns to string so that excel doesn't convert them into timestamps
full_data['publish_date'] = full_data['publish_date'].astype(str)
full_data['expire_date'] = full_data['expire_date'].astype(str)

# Make the extracted columns in lower case for better matching
for col in full_data.columns[0:2]:
    full_data[col] = full_data[col].str.lower().str.strip()


# Using fuzzy matching create two columns based on the best match
# between our extracted ports and port names in the back end, To get rid of misspelling problem
def find_best_match(value, choices, threshold=70):
    match = process.extractOne(value, choices, score_cutoff=threshold)
    return match[0] if match else None  # Return the first element if match is not None, otherwise return None


full_data['best_from'] = full_data['full_from'].apply(lambda x: find_best_match(x, ports['Port Name']))
full_data['best_to'] = full_data['full_to'].apply(lambda x: find_best_match(x, ports['Port Name']))
full_data5 = full_data

# make the following columns in lower case for better merging
ports['City'] = ports['City'].str.lower()
ports['Port Name'] = ports['Port Name'].str.lower()
full_data['best_from'] = full_data['best_from'].str.lower()
full_data['best_to'] = full_data['best_to'].str.lower()

# Check for ports doesn't exist in backend 
wrong_departure =  full_data5[(full_data5['best_from'].isnull()) \
                              & ~(full_data5['full_from'].isin(ports['Port Name'])) \
                                & ~(full_data5['full_from'].isin(ports['City']))]['full_from'].unique()
wrong_arrival = full_data5[(full_data5['best_to'].isnull()) \
                           & ~(full_data5['full_to'].isin(ports['Port Name'])) \
                            & ~(full_data5['full_to'].isin(ports['City']))]['full_to'].unique()

wrong_ports_dep = full_data[full_data['full_from'].isin(wrong_departure)]
wrong_ports_arr =  full_data[full_data['full_to'].isin(wrong_arrival)]

wrong_ports_dep['Error Type'] = 'Wrong departure port'
wrong_ports_arr['Error Type'] = 'Wrong arrival port'

wrong_ports_dep = wrong_ports_dep[['corrected_country', 'full_from', 'full_to', 'trip_type',  'Error Type']]
wrong_ports_arr = wrong_ports_arr[['corrected_country', 'full_from', 'full_to', 'trip_type',  'Error Type']]

# Check for countries the doesn't exist in backend
wrong_country = full_data1[full_data1['country'].isin(full_data2[full_data2['corrected_country'] == '']['country_x'].unique())]
wrong_country['Error Type'] = 'Wrong Country name'
wrong_country = wrong_country[['country', 'full_from', 'full_to', 'trip_type', 'Error Type']]

# Correct trips that entered city name instead of port name in the departure port
dep_cor1 = full_data.merge(ports, left_on='best_from', right_on = 'Port Name')
dep_cor2 = full_data.merge(ports, left_on='full_from', right_on ='City')
full_data = pd.concat([dep_cor1, dep_cor2], ignore_index=True)

# Correct trips that entered city name instead of port name in the arrival port
arr_cor1 = full_data.merge(ports, left_on='best_to', right_on = 'Port Name')
arr_cor2 = full_data.merge(ports, left_on='full_to', right_on ='City')
full_data = pd.concat([arr_cor1, arr_cor2], ignore_index=True)

# Extract and correct column names to the desired names 
full_data.columns
full_data.columns = ['full_from', 'full_to', 'trip_type', 'price', 'corrected_country',
       'Country', 'country_id', 'publish_date', 'expire_date', 'best_from',
       'best_to', 'City_from', 'Port_name_from', 'Port_code_from', 'port_from', 'City_to',
       'Port_name_to', 'Port_code_to', 'port_to']

# Create the url column 
full_data['url'] = full_data['Port_code_from'].str.lower() + '-' + full_data['Port_code_to'].str.lower() + '-' + full_data['trip_type'].str.lower()



# Delete cities that contains more than one port but mentioned in our data as a port which will cause confusion
grouped_cities = ports.groupby('City', ).count().iloc[:, 0]
cities_count = grouped_cities[grouped_cities > 1].index
unique_cities_departure = dep_cor2['City'].unique()
unique_cities_arrival = arr_cor2['City_y'].unique()
cities_with_multiple_ports_departue = [x for x in unique_cities_departure if x in   cities_count]
cities_with_multiple_ports_arrival = [x for x in unique_cities_arrival if x in   cities_count]

full_data = full_data[~full_data['full_from'].isin(cities_with_multiple_ports_departue)]
full_data = full_data[~full_data['full_to'].isin(cities_with_multiple_ports_arrival)]

# Delete trips with wrong country names
full_data = full_data[full_data['corrected_country'] != '']


# CREATE THE OUTPUT DATAFRAMES 
offers_template = full_data[['port_from', 'port_to', 'trip_type', 'url', 'publish_date', 'expire_date']]
offers_template = offers_template.drop_duplicates()
offer_contents = full_data[['url','country_id', 'price']]


# Save the output xlsx files
try:
    offers_template.to_excel('offers_template.xlsx', index=False)
    offer_contents.to_excel('offer_contents.xlsx', index=False)
except PermissionError:
    print('Remove old Output files')




# Trips with misspelled departure port or city instead of port
trips_with_misspelled_departure = full_data[full_data['best_from'] != full_data['full_from']][['full_from','Port_name_from', 'full_to', 'trip_type', 'Country']]
trips_with_misspelled_departure.columns = ['extracted_departure_port','correct_departue_port', 'extracted_arrival_port', 'trip_type', 'Country']
trips_with_misspelled_departure['Error Type'] = 'Misspelled departure port'
trips_with_misspelled_departure = trips_with_misspelled_departure[['Country', 'extracted_departure_port', 'extracted_arrival_port', 'trip_type',  'Error Type']]


# Trips with misspelled arrival port or city instead of port
trips_with_misspelled_arrive = full_data[full_data['best_to'] != full_data['full_to']][['full_from', 'full_to','Port_name_to', 'trip_type', 'Country']]
trips_with_misspelled_arrive.columns = ['extracted_departure_port', 'extracted_arrival_port','correct_arrival_port', 'trip_type', 'Country']
trips_with_misspelled_arrive['Error Type'] = 'Misspelled arrival port'
trips_with_misspelled_arrive = trips_with_misspelled_arrive[['Country', 'extracted_departure_port', 'extracted_arrival_port', 'trip_type',  'Error Type']]



report_dfs = [wrong_country, wrong_ports_dep, wrong_ports_arr, trips_with_misspelled_departure, trips_with_misspelled_arrive]
for df in report_dfs:
    df.columns = ['country', 'full_from', 'full_to', 'trip_type', 'Error Type']
pd.concat(report_dfs).to_excel('report.xlsx', index=False)